In [ ]:
rm -rf /kaggle/working/*

In [1]:
!pip install -q -U google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.0/200.0 kB 5.9 MB/s eta 0:00:00


In [6]:
!git clone --branch main https://github.com/giankev/Ancient-to-Modern-Italian-Automatic-Translation.git

Cloning into 'Ancient-to-Modern-Italian-Automatic-Translation'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 195 (delta 103), reused 94 (delta 34), pack-reused 0 (from 0)
Receiving objects: 100% (195/195), 727.44 KiB | 8.36 MiB/s, done.
Resolving deltas: 100% (103/103), done.


In [4]:
from kaggle_secrets import UserSecretsClient
import huggingface_hub
import os

try:
    user_secrets = UserSecretsClient()
    API_KEY = user_secrets.get_secret("apii")
    print("API KEY Loaded")
except Exception as e:
    print(f"Error during programmatic login: {e}")
    print("Please ensure your HF_TOKEN secret is correctly set in Kaggle.")

API KEY Loaded


In [9]:
from time import sleep

In [7]:
import pandas as pd

csv_translation = "/kaggle/working/Ancient-to-Modern-Italian-Automatic-Translation/Minerva350M_translations/CulturalIA-hw2_transl_Minerva350M_finetuned.csv"
df = pd.read_csv(csv_translation)

In [10]:
df.head()

,input,finetuned_translation
0,quella guerra ben fatta l' opera perché etc. E...,qu quella guerra ben fatta che ha fatto dell'A...
1,"crudele, e di tutte le colpe pigli vendetta, c...","e fa'it e' in modo di fare dazz, come dice il ..."
2,Non d' altra forza d' animo fue ornato Ponzio ...,"Trans aubit fou aubit fou aubit fou aufidiano,..."
3,Se questo piace a tutti e se 'l tempo hae biso...,"Se il tempo ha 'ra i segni dei feudiani, se no..."
4,Officio di questa arte pare che sia dicere app...,O or O or O or O or O or O or O or O or O or O...


In [11]:
from google import genai

client = genai.Client(api_key=API_KEY)

In [12]:
def prompt_evaluation(original: str, translated: str):
    prompt =  f"""Valuta la seguente traduzione dall'italiano antico all'italiano moderno.
                Concentrati sulla fedeltà semantica e la correttezza grammaticale.
                Fornisci un punteggio da 1 a 5 basandoti sulla seguente rubrica:
                1. Traduzione completamente inaccettabile: la traduzione non ha pertinenza con il significato originale, la frase generata è incomprensibile (gibberish) o senza senso.
                2. Errori semantici gravi, omissioni o aggiunte sostanziali rispetto alla frase originale. Gli errori sono di natura semantica e sintattica. È qualcosa che un umano non scriverebbe mai.
                3. Traduzione parzialmente errata: la traduzione è mediocre, contiene errori, ma sono per lo più errori minori, come refusi o piccoli errori semantici.
                4. Buona traduzione: la traduzione è per lo più corretta, sostanzialmente fedele al testo originale, ma lo stile potrebbe non corrispondere perfettamente alla frase originale; è comunque fluente, comprensibile e semanticamente accettabile.
                5. Traduzione perfetta: la traduzione è accurata, fluente, completa e coerente. Ha mantenuto il significato originale il più possibile.
                fornisci solo il numero del punteggio (da 1 a 5).
                frase originale: "{original}"
                frase tradotta: "{translated}"
                
                """
    return prompt

In [13]:
results = []
for index, row in df.iterrows():
    
    original = row["input"]
    translation = row["finetuned_translation"]
    prompt = prompt_evaluation(original, translation)

    response = client.models.generate_content(
             model="gemini-2.0-flash-lite",
             contents=prompt
    )

    results.append({
        "sentence": original,
        "translation": translation,
        "score": response.text
    })

    sleep(2) 

In [14]:
df_results = pd.DataFrame(results)
df_results.to_csv("/kaggle/working/evaluation_minerva350M_finetuned.csv", index=False, encoding='utf-8')

In [15]:
import json

with open("/kaggle/working/evaluation_minerva350M_finetuned.csv.jsonl", "w", encoding="utf-8") as f:
    for entry in results:
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')